In [115]:
from surprise import SVD
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold
from surprise import accuracy

import pandas as pd
import os
import numpy as np

In [53]:
# Get uid with >100 ratings
folder = "C:\\Users\\yjin9\\Documents\\python-scripts\\data_incubator\\data_scraping_cleaning\\top_rated\\" 
user_file_path =os.path.join(folder,"uid_count.csv")
user_df=pd.read_csv(user_file_path,encoding="utf-8")
uid_list=user_df[user_df.uid_count>100]["uid"].apply(lambda x:str(x))

In [82]:
# Get combined score dataframe 
summary_file_path =os.path.join(folder,"beer_summary.csv")
beer_ids=pd.read_csv(summary_file_path,encoding="utf-8")["beer_ids"].tolist()

try:
    score_df=pd.read_csv(os.path.join(folder,"combined_ba_score.csv"),encoding="utf-8")[["user_id","item","ba_score"]]

except FileNotFoundError:
    score_df=pd.DataFrame({"user_id":[],"ba_score":[],"item":[]})

    for beer_idx in range(len(beer_ids)):
        file_path =os.path.join(folder,"beer_"+str(beer_ids[beer_idx])+".csv")
        beer_ratings=pd.read_csv(file_path,encoding="utf-8")
        beer_ratings=beer_ratings[~np.isnan(beer_ratings.user_id)][["user_id","ba_score"]]
        beer_ratings.user_id=beer_ratings.user_id.apply(lambda x: str(int(x)))
        beer_ratings=beer_ratings[beer_ratings.user_id.isin(uid_list)]
        beer_ratings["item"]=str(beer_ids[beer_idx])
        score_df=score_df.append(beer_ratings)
    score_df.to_csv(os.path.join(folder,"combined_ba_score.csv"),encoding="utf-8")

In [107]:
# Total # of users
len(uid_list)

8999

In [92]:
# Total # of ratings
score_df.shape

(2221464, 3)

In [113]:
# Total # of beer items
len(beer_ids)

1337

In [87]:
reader = Reader(rating_scale=(1.0, 5.0))
data = Dataset.load_from_df(score_df[['user_id', 'item', 'ba_score']], reader)
algo = SVD()
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.3713  0.3713  0.3715  0.3713  0.3710  0.3713  0.0001  
MAE (testset)     0.2658  0.2664  0.2668  0.2665  0.2665  0.2664  0.0003  
Fit time          115.71  119.29  119.42  115.83  113.43  116.74  2.30    
Test time         6.72    5.40    5.77    5.02    5.69    5.72    0.56    


{'fit_time': (115.7073724269867,
  119.29231429100037,
  119.42065668106079,
  115.82809925079346,
  113.42971754074097),
 'test_mae': array([ 0.26583956,  0.26635213,  0.26676392,  0.26648657,  0.26653371]),
 'test_rmse': array([ 0.37130505,  0.37129908,  0.3714541 ,  0.37127379,  0.37103955]),
 'test_time': (6.715921401977539,
  5.399384021759033,
  5.767340898513794,
  5.015340328216553,
  5.687127351760864)}

In [118]:
algo = SVD()
kf = KFold(n_splits=3)

for trainset, testset in kf.split(data):
    algo.fit(trainset)
    predictions = algo.test(testset)
    accuracy.rmse(predictions, verbose=True)

RMSE: 0.3743
RMSE: 0.3733
RMSE: 0.3733
